<a href="https://colab.research.google.com/github/valogonor/Hacker-News-Saltiness/blob/master/HackerNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.cloud import bigquery

# For Colab we can have it in Google Drive

from google.colab import drive

drive.mount('/content/drive')

%env GOOGLE_APPLICATION_CREDENTIALS=/content/drive/My Drive/[environment_variables_here]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: GOOGLE_APPLICATION_CREDENTIALS=/content/drive/My Drive/hackernews-234220-f79de4ee5df0.json


In [0]:
client = bigquery.Client()

In [0]:
hn_dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')

In [0]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [0]:
hn_dset = client.get_dataset(hn_dataset_ref)

In [0]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [0]:
hn_comments = client.get_table(hn_dset.table('comments'))

In [0]:
type(hn_comments)

google.cloud.bigquery.table.Table

In [0]:
[command for command in dir(hn_comments) if not command.startswith('_')]

['clustering_fields',
 'created',
 'dataset_id',
 'description',
 'encryption_configuration',
 'etag',
 'expires',
 'external_data_configuration',
 'friendly_name',
 'from_api_repr',
 'from_string',
 'full_table_id',
 'labels',
 'location',
 'modified',
 'num_bytes',
 'num_rows',
 'partition_expiration',
 'partitioning_type',
 'path',
 'project',
 'reference',
 'schema',
 'self_link',
 'streaming_buffer',
 'table_id',
 'table_type',
 'time_partitioning',
 'to_api_repr',
 'to_bqstorage',
 'view_query',
 'view_use_legacy_sql']

In [0]:
hn_comments.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

In [0]:
schema_subset = [col for col in hn_comments.schema if col.name in ('by', 'text', 'time_ts')]
results = [x for x in client.list_rows(hn_comments, selected_fields=schema_subset, max_results=10)]

In [0]:
print(results)

[Row(('5l', datetime.datetime(2011, 6, 27, 14, 28, 1, tzinfo=<UTC>), 'And the glazier who fixed all the broken windows also left his money to good causes.'), {'by': 0, 'time_ts': 1, 'text': 2}), Row(('99', datetime.datetime(2013, 6, 3, 4, 34, 8, tzinfo=<UTC>), 'Does canada have the equivalent of H1B/Green card for work sponsorship? What do you think of that?'), {'by': 0, 'time_ts': 1, 'text': 2}), Row(('AF', datetime.datetime(2007, 5, 12, 17, 53, 20, tzinfo=<UTC>), 'Speaking of Rails, there are other options in the Python world besides Django.<p>Pylons is a very Rails-y framework with the difference being that it is made to be easy to customize. In Rails if you don\'t like something you are going to have a hard time changing it out unless you are a good hacker. In Pylons that is easy, and you\'ve got access to Python\'s vastly better platform (speed, Unicode support) and libraries.<p>If you are an absolute beginning programmer it might be kind of hard to pick up, but if you\'ve program

In [0]:
for i in results:
    print(dict(i))

{'by': '5l', 'time_ts': datetime.datetime(2011, 6, 27, 14, 28, 1, tzinfo=<UTC>), 'text': 'And the glazier who fixed all the broken windows also left his money to good causes.'}
{'by': '99', 'time_ts': datetime.datetime(2013, 6, 3, 4, 34, 8, tzinfo=<UTC>), 'text': 'Does canada have the equivalent of H1B/Green card for work sponsorship? What do you think of that?'}
{'by': 'AF', 'time_ts': datetime.datetime(2007, 5, 12, 17, 53, 20, tzinfo=<UTC>), 'text': 'Speaking of Rails, there are other options in the Python world besides Django.<p>Pylons is a very Rails-y framework with the difference being that it is made to be easy to customize. In Rails if you don\'t like something you are going to have a hard time changing it out unless you are a good hacker. In Pylons that is easy, and you\'ve got access to Python\'s vastly better platform (speed, Unicode support) and libraries.<p>If you are an absolute beginning programmer it might be kind of hard to pick up, but if you\'ve programmed a bit or y

In [0]:
# Using WHERE reduces the amount of data scanned / quota used
query = """
SELECT author, COUNT(author)
FROM `bigquery-public-data.hacker_news.comments`
GROUP BY author
HAVING COUNT(author) >= 5
ORDER BY COUNT(author) DESC
"""

In [0]:
query_job = client.query(query)

In [0]:
iterator = query_job.result(timeout=1)
commenters = list(iterator)
commenters[:10]

[Row(('tptacek', 33839), {'author': 0, 'f0_': 1}),
 Row(('jacquesm', 21107), {'author': 0, 'f0_': 1}),
 Row(('jrockway', 13626), {'author': 0, 'f0_': 1}),
 Row(('DanBC', 12902), {'author': 0, 'f0_': 1}),
 Row(('anigbrowl', 11395), {'author': 0, 'f0_': 1}),
 Row(('rayiner', 11080), {'author': 0, 'f0_': 1}),
 Row(('sp332', 10882), {'author': 0, 'f0_': 1}),
 Row(('davidw', 10764), {'author': 0, 'f0_': 1}),
 Row(('dragonwriter', 10723), {'author': 0, 'f0_': 1}),
 Row(('rbanffy', 10557), {'author': 0, 'f0_': 1})]

In [0]:
len(commenters)

85350

In [0]:
query = """
SELECT author, text, time_ts
FROM `bigquery-public-data.hacker_news.comments`
"""

In [0]:
query_job = client.query(query)

In [0]:
iterator = query_job.result(timeout=30)
rows = list(iterator)
rows[:5]

[Row(('spenvo', 'So this has been going on since at least 2008... Extremely disappointing but not surprising.[0]  What Americans forget is that GHCQ&#x27;s policies affect us as much or more than the NSA&#x27;s and that by &#x27;backdoor-ing&#x27; crypto, as Cameron &amp; Obama want, we forfeit our privacy to foreign governments as well.<p>How can democratic society continue to function if the state indulges its interest in journalists&#x27; and attorneys&#x27; confidential communications without any inhibitions?<p>This demonstrates that Cameron&#x27;s pledge to only access laypersons&#x27; communication in the most &quot;dire circumstances&quot; (the case he makes for backdooring crypto) is pure rhetoric.<p>[0] <a href="https://firstlook.org/theintercept/2014/11/06/uk-surveillance-of-lawyers-journalists-gchq/" rel="nofollow">https:&#x2F;&#x2F;firstlook.org&#x2F;theintercept&#x2F;2014&#x2F;11&#x2F;06&#x2F;uk-surveillanc...</a>', datetime.datetime(2015, 1, 19, 19, 30, 39, tzinfo=<UTC>))

In [0]:
len(rows)

8399417

In [0]:
import numpy as np
import pandas as pd

In [0]:
df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

In [0]:
df.head(10)

,author,text,time_ts
0,spenvo,So this has been going on since at least 2008....,2015-01-19 19:30:39+00:00
1,splawn,Good point! More stay at home parents is anoth...,2015-07-02 14:18:37+00:00
2,spodek,"Many posts here critical of JFK so far, includ...",2013-10-23 23:28:32+00:00
3,sprout,They don't want a third-party app to become a ...,2010-06-11 19:59:58+00:00
4,spxdcz,"Oh, good idea. I might do a follow-up/edit wit...",2010-01-05 14:53:39+00:00
5,sqrt17,This is precisely why no one uses the raw wave...,2010-05-03 07:59:22+00:00
6,ssijak,Thanks for the reply! I implemented some of th...,2014-04-21 09:54:25+00:00
7,ssmoot,Right on the money.<p>When your boss asks a qu...,2014-03-14 16:41:43+00:00
8,ssmoot,I've done this (GMail setup). Didn't work for ...,2011-08-19 18:56:05+00:00
9,ssmoot,&#62; JRuby as a gateway drug to the Land of R...,2011-06-08 03:34:57+00:00


In [0]:
import re
from textblob import TextBlob

def clean_text(text):
    """
    Utility function to clean text by removing links, special characters
    using simple regex statements. May not be needed.
    """
    return ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\ / \ / \S+)","", text))


def get_sentiment(text):
    """
    Utility function to classify sentiment of passed text
    using textblob's sentiment method. Return the polarity
    score as a float within the range [-1.0, 1.0]
    """
    # create TextBlob object of passed text's polarity
    return TextBlob(text).sentiment.polarity

In [0]:
df.isna().sum()

author     227736
text       227740
time_ts         0
dtype: int64

In [0]:
df_no_nulls = df.dropna()
df_no_nulls.isna().sum()

author     0
text       0
time_ts    0
dtype: int64

In [0]:
df_no_nulls.isna().sum()

author     0
text       0
time_ts    0
dtype: int64

In [0]:
df_no_nulls.shape

(8171677, 3)

In [0]:
df_no_nulls.to_csv('hackernews.csv', index=False)

In [0]:
df_fourth1 = df_no_nulls[:2042920]
df_fourth1.shape

(2042920, 3)

In [0]:
df_fourth1.to_csv('hackernews1.csv', index=False)

In [35]:
df_eighth1 = df_no_nulls[:1021460]
df_eighth1.shape

(1021460, 3)

In [0]:
df_eighth1.to_csv('hackernews1.csv', index=False)